In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import io
import matplotlib.pyplot as plt
from matplotlib import image
from pathlib import Path
import os.path
from PIL import Image


In [ ]:
def load_imgs(img_path):
    imgs=[]
    label=[]
    label1=os.listdir(img_path)
    for index in label1:
        label2 = os.listdir(img_path+'/' + index)
        for index2 in label2:
            c=0
            img=Image.open(img_path+'/' + index + '/' + index2).convert('RGB')
            img=img.resize(size=(64,64))
            #img=img.convert('L')
            img=np.array(img,dtype=np.float16)/255
            imgs.append(np.array(img)) #.flatten())
            label.append(index)
            del img
    return np.array(imgs),label

In [ ]:
X_train, y_train=load_imgs('/kaggle/input/vegetable-image-dataset/Vegetable Images/train')

In [ ]:
X_test, y_test=load_imgs('/kaggle/input/vegetable-image-dataset/Vegetable Images/test')

In [ ]:
X_val, y_val=load_imgs('/kaggle/input/vegetable-image-dataset/Vegetable Images/validation')

In [ ]:
X_train.shape

In [ ]:
#카테고리형 시리즈 만들고 레이블화
#훈련
X_train=np.array(X_train)
y_train=pd.Series(y_train,dtype='category')
#테스트
X_test=np.array(X_test)
y_test=pd.Series(y_test,dtype='category')
#검증
X_val=np.array(X_val)
y_val=pd.Series(y_val,dtype='category')
#범주화
y_train=y_train.cat.codes
y_test=y_test.cat.codes
y_val=y_val.cat.codes

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, Conv2D, MaxPooling2D, BatchNormalization,MaxPool2D,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
#논문의 있는 VGGNET 구조
model = Sequential()
# model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, 3, strides= 1, padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, 3, strides= 1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2,2), strides=2))

model.add(Conv2D(128, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(512, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(512, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(512, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2, 2), strides=2))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))
model.summary()

In [ ]:
from tensorflow import keras

#성능 안오르면 멈춤
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
hist = model.fit(
    X_train,y_train,
    validation_data=(X_val,y_val),
    epochs=50, verbose=0
)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
res = model.predict(X_test)


In [ ]:
list_res = []
for i in range(len(res)):
    list_res.append(np.argmax(res[i]))

In [ ]:
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve


y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred, axis=-1)
print(y_pred)
fper, tper, thresholds = roc_curve(y_test, y_pred, pos_label=14)
plot_roc_curve(fper, tper)